In [25]:
from analysis2014.process_2014_data import get_image_vec_list
item_dict = get_image_vec_list()
item_mat = list(item_dict.values())
num_item_feat = len(item_mat[0])



processed 1000 asins
processed 2000 asins
processed 3000 asins
processed 4000 asins
processed 5000 asins
processed 6000 asins


In [26]:
from analysis2014.process_2014_data import get_mapped_ratings
import pandas as pd
import json
with open("analysis2014/baby_5_core_reviews.json", "r") as pvr_file:
    pv_rev = json.load(pvr_file)
df = pd.DataFrame(pv_rev)

In [27]:
counter = 0
r_id_r_int_mapping = {}
for r_id in pd.unique(df["reviewer_id"]):
    if r_id not in r_id_r_int_mapping:
        r_id_r_int_mapping[r_id] = counter
        counter += 1

In [28]:
counter = 0
p_id_p_int_mapping = {}
for p_id in pd.unique(df["asin"]):
    if p_id not in p_id_p_int_mapping:
        p_id_p_int_mapping[p_id] = counter
        counter += 1

In [29]:
import json
with open("analysis2014/reviews_products.json", "w") as rp:
    json.dump(p_id_p_int_mapping, rp)

with open("analysis2014/reviews_customers.json", "w") as rr:
    json.dump(r_id_r_int_mapping, rr)

In [30]:
df["reviewer_id"] = df["reviewer_id"].map(r_id_r_int_mapping)
df["item_id"] = df["asin"].map(p_id_p_int_mapping)


In [31]:
from cdl_model import CollaborativeDeepLearning
model = CollaborativeDeepLearning(item_mat, [num_item_feat, 512, 64], item_dict)

In [32]:
# This should only be done once
model.pretrain(lamda_w=0.001, encoder_noise=0.3, epochs=10)
trained_encoders = model.trained_encoders
trained_decoders = model.trained_decoders

Epoch 1/10
110/110 - 6s - loss: 1.3424
Epoch 2/10
110/110 - 5s - loss: 1.2269
Epoch 3/10
110/110 - 5s - loss: 1.2268
Epoch 4/10
110/110 - 5s - loss: 1.2268
Epoch 5/10
110/110 - 5s - loss: 1.2268
Epoch 6/10
110/110 - 5s - loss: 1.2267
Epoch 7/10
110/110 - 5s - loss: 1.2267
Epoch 8/10
110/110 - 5s - loss: 1.2269
Epoch 9/10
110/110 - 5s - loss: 1.2270
Epoch 10/10
110/110 - 5s - loss: 1.2268
Epoch 1/10
110/110 - 1s - loss: 0.0913
Epoch 2/10
110/110 - 0s - loss: 0.0246
Epoch 3/10
110/110 - 0s - loss: 0.0246
Epoch 4/10
110/110 - 0s - loss: 0.0246
Epoch 5/10
110/110 - 0s - loss: 0.0246
Epoch 6/10
110/110 - 0s - loss: 0.0246
Epoch 7/10
110/110 - 0s - loss: 0.0245
Epoch 8/10
110/110 - 0s - loss: 0.0246
Epoch 9/10
110/110 - 0s - loss: 0.0246
Epoch 10/10
110/110 - 0s - loss: 0.0246


In [33]:
model.trained_encoders = trained_encoders
model.trained_decoders = trained_decoders

In [34]:
train_mat = df.sample(frac=0.8,random_state=200) #random state is a seed value
test_mat = df.drop(train_mat.index)
model_history = model.fineture(train_mat, test_mat, lamda_u=0.01, lamda_v=0.1, lamda_n=0.1, lr=0.01, epochs=3)
testing_rmse = model.getRMSE(test_mat)
print('Testing RMSE = {}'.format(testing_rmse))

Epoch 1/3
2010/2010 [==============================] - 241s 118ms/step - loss: 29824.5616 - dot_layer_loss: 29.4255 - model_10_loss: 1.2698 - val_loss: 659.3342 - val_dot_layer_loss: 0.7727 - val_model_10_loss: 1.2764

In [35]:
test_user, test_item, test_item_feat, test_label = model.matrix2input(test_mat.iloc[:,])

In [36]:
pred_out = model.cdl_model.predict([test_user, test_item, test_item_feat])
print(test_label.flatten())
print(pred_out[0].flatten())

[1.  0.8 1.  ... 0.8 1.  1. ]
[ 1.4216048e-01  3.0293725e-02  2.1787889e-02 ... -3.8878259e-13
  4.4018678e-02  3.0609129e-02]


In [45]:
test_labels_flat = test_label.flatten()
rating_predictions = pred_out[0].flatten()
tuples = [(test_labels_flat[i], rating_predictions[i]) for i in range(len(test_labels_flat))]

for i in [30, 50, 100, 300]:
    recommendations = sorted(tuples, key=lambda x: x[1], reverse=True)[:i]
    prediction_average = sum([x[0] for x in recommendations])/len(recommendations)
    dataset_mean = test_labels_flat.mean()
    print("The average rating of the top {} recommended products is: {}".format(i, round(prediction_average, 2)))
    print("The average rating of the entire test dataset is: ", round(dataset_mean, 2))
    print("The improvement percentage is: {}% \n".format(round((prediction_average-dataset_mean)*100), 5))

The average rating of the top 30 recommended products is: 0.93
The average rating of the entire test dataset is:  0.84
The improvement percentage is: 9% 

The average rating of the top 50 recommended products is: 0.89
The average rating of the entire test dataset is:  0.84
The improvement percentage is: 5% 

The average rating of the top 100 recommended products is: 0.87
The average rating of the entire test dataset is:  0.84
The improvement percentage is: 3% 

The average rating of the top 300 recommended products is: 0.87
The average rating of the entire test dataset is:  0.84
The improvement percentage is: 3% 

